In [1]:
import tensorflow as tf

/Users/esteban/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/esteban/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import csv

def create_label_dict():
    labels = {}
    with open('trainLabels.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            if row[0] != 'Id':
                labels[row[0].replace("\'","")] = int(row[1])
    return labels



In [3]:
import numpy as np
import math
import os
from PIL import Image

path = '/Users/esteban/malware-data/data'

def generate_images(size,extension, save_path):
    directory = os.listdir(path)
    for x in directory:
        digits = [] 
        if x.endswith('.bytes'):
            full_path = os.path.join(path, x)
            with open(full_path) as f:
                for line in f:
                    l = line.split()
                    a = l[1:]
                
                    for c in a:
                        if c != '??':
                            bin = int(c,16)
                            digits.append(bin)
                        else:
                            digits.append(0)
        
            #---create array from digits list
            larray = np.asarray(digits)

            #---determines number of coordinates of array/ finds the square and rounds up/ reshapes the dimensions
            length = len(larray)       
            q = math.sqrt(length)
            dim = math.ceil(q)
            shape = (dim, dim)
            larray.resize(shape)
            #print(larray.shape)
        
            #--normalize matrix
            normalized = (larray-np.min(larray)) / (np.max(larray)-np.min(larray))         
        
            #--create an image using pillows library
            img = Image.fromarray(normalized)
        
            #--create 28x28 image- sav
            img_name = (str(x[0:20]) + extension + '.tiff')
            img = img.resize((size, size), Image.ANTIALIAS)
            img.save(os.path.join(save_path, img_name))
        

In [4]:
generate_images(28,'_28x28', '/Users/esteban/malware-data/data/28x28')


In [5]:
generate_images(224,'_224x224', '/Users/esteban/malware-data/data/224x224')

In [6]:
generate_images(1000,'_1000x1000', '/Users/esteban/malware-data/data/1000x1000')

In [7]:
def extract_file_name(path_to_file):
    base_name = os.path.basename(path_to_file)
    return base_name[:20]

In [8]:
from random import shuffle
import glob
shuffle_data = True  # shuffle the addresses before saving
image_path = '/Users/esteban/malware-data/data/224x224/*.tiff'

label_dict = create_label_dict()
# read addresses and labels from the 'train' folder
addrs = glob.glob(image_path)
labels = [label_dict[extract_file_name(addr)] for addr in addrs]  
# to shuffle data
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)
    
# Divide the hata into 60% train, 20% validation, and 20% test
train_addrs = addrs[0:int(1.0*len(addrs))]
train_labels = labels[0:int(1.0*len(labels))]
#val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
#val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]
#test_addrs = addrs[int(0.8*len(addrs)):]
#test_labels = labels[int(0.8*len(labels)):]

In [9]:
print (extract_file_name('/Users/esteban/malware-data/data/224x224/0A32eTdBKayjCWhZqDOQ_224x224.tiff'))

0A32eTdBKayjCWhZqDOQ


In [10]:
print(labels)

(2, 7)


In [11]:
print(train_addrs)

('/Users/esteban/malware-data/data/224x224/0A32eTdBKayjCWhZqDOQ_224x224.tiff', '/Users/esteban/malware-data/data/224x224/0ACDbR5M3ZhBJajygTuf_224x224.tiff')


In [32]:
import cv2
from matplotlib import pyplot as plt

def load_image(addr):
    
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.imread(addr,-1)
    
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.float32)
    
    return img

In [33]:

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [34]:
import sys

train_filename = 'train.tfrecords'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(train_addrs)):
    # print how many images are saved every 1000 images
    if not i % 1000:
        print ('Train data: {}/{}'.format(i, len(train_addrs)))
        sys.stdout.flush()
    # Load the image
    img = load_image(train_addrs[i])
    label = train_labels[i]
    # Create a feature
    feature = {'train/label': _int64_feature(label),
               'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring()))}
    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    
    # Serialize to string and write on the file
    writer.write(example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/2


In [40]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
data_path = 'train.tfrecords'  # address to save the hdf5 file
with tf.Session() as sess:
   
    feature = {'train/image': tf.FixedLenFeature([], tf.string),
               'train/label': tf.FixedLenFeature([], tf.int64)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path], num_epochs=4)
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['train/image'], tf.float32)
    
    # Cast label data into int32
    label = tf.cast(features['train/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [224, 224, 1])
    
    
    # Any preprocessing here ...
   
    
    # Creates batches by randomly shuffling tensors
    images, labels = tf.train.shuffle_batch([image, label], batch_size=1, capacity=30, num_threads=1, min_after_dequeue=10)
    
    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for batch_index in range(1):
        img, lbl = sess.run([images, labels])
        print (lbl)
        
    # Stop the threads
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)
    sess.close()

[7]
